https://python.langchain.com/docs/modules/agents/how_to/custom_multi_action_agent


In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [25]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent
import langchain

langchain.debug = True

In [27]:
def random_word(query: str) -> str:
    print(f"\nNow i'm doing this with this querty {query}")
    return "Foo"

In [26]:
search = SerpAPIWrapper()
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for when you need answer questions about current events or facts.",
)

random_word = Tool(
    name="RandomWord", func=random_word, description="call this to ge a random word."
)
tools = [search_tool, random_word]

In [35]:
from typing import List, Tuple, Any, Union
from langchain.schema import AgentAction, AgentFinish


class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.
        Args:
            intermediate_steps: Steps the LLM has taken to date, along with observations
            ** kwargs: User inputs.
        Returns:
            Actions specifying what tool to use.
        """
        print("intermediate_steps", intermediate_steps)
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(
                return_values={"output": "intermediate_step length > 0"}, log=""
            )

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        print("intermediate_steps", intermediate_steps)
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

In [34]:
agent = FakeAgent()
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [36]:
agent_executor.run("How many people live in canada as of 2023?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "How many people live in canada as of 2023?"
}
[]
[tool/start] [1:chain:AgentExecutor > 2:tool:Search] Entering Tool run with input:
"How many people live in canada as of 2023?"
[tool/end] [1:chain:AgentExecutor > 2:tool:Search] s] Exiting Tool run with output:
"38,781,291 people"
[tool/start] [1:chain:AgentExecutor > 3:tool:RandomWord] Entering Tool run with input:
"How many people live in canada as of 2023?"
[tool/start] [1:chain:AgentExecutor > 3:tool:RandomWord > 4:tool:RandomWord] Entering Tool run with input:
"How many people live in canada as of 2023?"

Now i'm doing this with this querty How many people live in canada as of 2023?
[tool/end] [1:chain:AgentExecutor > 3:tool:RandomWord > 4:tool:RandomWord] s] Exiting Tool run with output:
"Foo"
[tool/end] [1:chain:AgentExecutor > 3:tool:RandomWord] s] Exiting Tool run with output:
"Foo"
[(AgentAction(tool='Search', tool_input='How many people live in

'intermediate_step length > 0'